In [ ]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
#1. count the orders under each status
base_order_rdd=spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")
base_order_map=base_order_rdd.map(lambda line: (line.split(",")[3],1))
base_order_reduce=base_order_map.reduceByKey( lambda a,b : a+b )
base_order_reduce.take(10)

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('COMPLETE', 22899),
 ('PENDING_PAYMENT', 15030),
 ('SUSPECTED_FRAUD', 1558),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729)]

In [12]:
#2. find the premium customers (Top 10 who placed the most number of orders)
premium_cust_rdd=base_order_rdd.map(lambda line : (line.split(",")[2],1 ))
premium_cust_reduce=premium_cust_rdd.reduceByKey(lambda a,b : a+b)
premium_top10_cust=premium_cust_reduce.sortBy(lambda a : a[1],False)
premium_top10_cust.take(10)

[('6316', 16),
 ('12431', 16),
 ('569', 16),
 ('5897', 16),
 ('5283', 15),
 ('12284', 15),
 ('5654', 15),
 ('221', 15),
 ('4320', 15),
 ('5624', 15)]

In [20]:
# 3. distinct count of customers who placed atleast one order
dist_map=base_order_rdd.map(lambda line : (line.split(",")[2],1)).distinct()
dist_map.count()


12405

In [38]:
# 4. which customers has the maximum number of CLOSED orders
# base_order_rdd.take(10)
closed_rdd_filter= base_order_rdd.filter(lambda line : line.split(",")[3]=='CLOSED')
closed_rdd_map=closed_rdd_filter.map(lambda line: (line.split(",")[2],1))
closed_rdd_reduce=closed_rdd_map.reduceByKey(lambda a,b : a+b)
closed_rdd_reduce.sortBy(lambda line :line[1],False).take(10)

[('1833', 6),
 ('5493', 5),
 ('1363', 5),
 ('1687', 5),
 ('2768', 4),
 ('5319', 4),
 ('437', 4),
 ('2774', 4),
 ('10111', 4),
 ('5582', 4)]